# Toy example for multi-genes and multi-CNVs in a region
## One or more causal genes

In [1]:
import numpy as np
import pandas as pd
import feather
from pandasql import sqldf
from scipy import stats
import sys
import fisher

In [26]:
def toy_multicnv_multicausal(p, n_cnv, causal, n_geom=10000, n_gene=10, const1=0.5, const2=0.05, seed=12):
    '''A region with multiple genes and CNVs. CNVs probably overlap. At least two causal genes.
    "causal" is a list of positions of causal genes.
    "n_gene" is the number of genes this region harbors.
    "n_cnv" is the number of CNV in this region. "n_geom" is the number of geometric variables generated.
    "p" is the probability of geometric dist, then randomly generate a certain number of geometric variables.
    I use these variables minus 1 as the length of CNVs, which means the number of genes that a CNV overlap with.
    The maximum length for CNV in this region is overlapping with 10 genes'''
    np.random.seed(seed)
    if n_cnv > n_geom:
        print ("The number of CNV is larger than the number of geometric numbers")
        sys.exit()
        return None
    geom_minus_1 = np.random.geometric(p, size=n_geom) - 1
    if n_cnv > len(geom_minus_1[geom_minus_1 <= n_gene]):
        print ("Cannot take a larger sample than population when 'replace=False'")
        sys.exit()
        return None
    cnv_len = np.random.choice(geom_minus_1[geom_minus_1 <= n_gene], n_cnv, replace=False)
    cnv_start_pos = [np.random.choice(range(n_gene+1-i)) if i!=0 else -1 for i in cnv_len]
    ptn_ls = []
    for j,i in enumerate(cnv_start_pos):
    ## j is the index of i in cnv start position list, so cnv_len[j] is the length of the corresponding cnv;
    ## i is the corresponding cnv start position
        if i == -1:
            ptn = [0]*n_gene
        else:
            ptn = [0]*i + [1]*cnv_len[j] + [0]*(n_gene - cnv_len[j] - i)
        ptn_ls.append(ptn)
    mat = [[np.random.normal(0,1)] + line for line in ptn_ls]
    config = []
    for line in mat:
        line[0] = line[0] + line[1:].count(1)*const2
        config.append(line[0])
#         only one causal gene, and gene pos is no more than the number of gene covered in this region
        if len(causal) == 1 and (causal[0] > len(line[1:]) or causal[0] <= 0):
            print ("Single causal CNV index out of range")
            sys.exit()
        elif len(causal) == 1 and causal[0] <= len(line[1:]):
            line[0] = line[0] + line[causal[0]]*const1
            config[-1] = line[0]
        elif max(causal) > len(line[1:]) or min(causal) <= 0:
            print ("Causal CNV index out of range")
            sys.exit()
        elif any(line[x] for x in causal):
            line[0] = line[0] + const1*[line[x] for x in causal].count(1)
            config[-1] = line[0]
        else:
            continue
    for line in mat:
        line[0] = 1 if line[0] >= np.median(config) else 0
    mat = np.matrix(mat)
    df = pd.DataFrame(mat, columns = ["phenotype"] + ["gene{}".format(i+1) for i in range(mat.shape[1]-1)])
    counts = df.groupby(["gene{}".format(i+1) for i in range(mat.shape[1]-1)] + ["phenotype"]).size()
    
    for col in df.columns:
        df[col] = df[col].astype(np.uint8)
    output = feather.write_dataframe(df, "data/toy_n{}_p{}_causal{}_const{}_{}.feather"
                                     .format(len(cnv_len), p, causal, const1, const2))
    return df, counts

In [27]:
df, counts = toy_multicnv_multicausal(p=0.5, n_cnv=5000, causal=[1,15], n_gene=15, const1=0.45, const2=0.03)
print (df["phenotype"].tolist().count(1))

2500


In [10]:
from utils import run_dap_lite
fileout = "data/toy_multi_causal_10.dap"
run_dap_lite(df, fileout, grid=[(0, -9.2)])

2017-09-05 18:23:23.243223
2017-09-05 18:23:38.362238


In [42]:
def get_summary_stats(gene_df, subst = -100):
    '''get summary statistics (e.g. t-statistic, p-value) to be used for TORUS, then transfer p-value to z-score
    '''
    cases = gene_df[gene_df["phenotype"]==1]
    ctrls = gene_df[gene_df["phenotype"]==0]
    res_dict = {}
    for gene in list(gene_df)[1:]:
        n_gene_case = len(cases[cases[gene]==1])
        n_nogene_case = len(cases[cases[gene]==0])
        n_gene_ctrl = len(ctrls[ctrls[gene]==1])
        n_nogene_ctrl = len(ctrls[ctrls[gene]==0])
        fisher_stats = fisher.pvalue(n_gene_case, n_gene_ctrl, n_nogene_case, n_nogene_ctrl)
        p_value_two = fisher_stats.two_tail
        p_value_left = fisher_stats.left_tail
        p_value_right = fisher_stats.right_tail
        print (p_value_two, p_value_left, p_value_right)
        if p_value_right < 0.5 and p_value_two < 1:
            z_score = stats.norm.ppf(1-p_value_right)
        elif p_value_left < 0.5 and p_value_two < 1:
            z_score = -stats.norm.ppf(1-p_value_left)
        else:
            z_score = subst
        res_dict[gene] = z_score
    return res_dict

In [29]:
get_summary_stats(df)

0.00024462176432881117 0.9999348800778008 0.00012231088216440556
0.9529633290053254 0.47648166450266255 0.5702296709879797
0.2208036361594937 0.1104018180797468 0.9091897147620988
0.6587783076749487 0.7095302789396232 0.32938915383747447
0.7860605670384375 0.6480298274140719 0.39303028351921887
0.11233726712005403 0.9545154603485689 0.056168633560027006
0.48575252342344866 0.24287626171172427 0.7894019985716844
0.4004777133244753 0.827899293102909 0.20023885666223779
0.48360579825626965 0.7905444120471704 0.24180289912813469
0.5922751819477777 0.7397794586927233 0.2961375909738887
0.9999999999985895 0.5445378127845576 0.49999999999929434
0.029128786223406904 0.9891858882638618 0.014564393111703445
0.006481393111004906 0.9977527840366609 0.003240696555502453
0.05360166558996012 0.9802540552225478 0.026800832794980065
9.854096454110435e-06 0.9999975943958207 4.927048227055219e-06


{'gene1': 3.6678257823421223,
 'gene10': 0.53554191477264124,
 'gene11': 1.7677085129515064e-12,
 'gene12': 2.1817391980275018,
 'gene13': 2.7223781458865797,
 'gene14': 1.9300417447140998,
 'gene15': 4.420350444922196,
 'gene2': -0.058985912204876305,
 'gene3': -1.2243939880406665,
 'gene4': 0.44160051996547306,
 'gene5': 0.27142969322658017,
 'gene6': 1.587774820610055,
 'gene7': -0.69708032974689282,
 'gene8': 0.84076836409598865,
 'gene9': 0.7005149080833758}

In [44]:
stats.norm.ppf((1+(1-0.9999999999985895))/2)

1.7677085129515064e-12

In [43]:
get_summary_stats(df)

0.00024462176432881117 0.9999348800778008 0.00012231088216440556
0.9529633290053254 0.47648166450266255 0.5702296709879797
0.2208036361594937 0.1104018180797468 0.9091897147620988
0.6587783076749487 0.7095302789396232 0.32938915383747447
0.7860605670384375 0.6480298274140719 0.39303028351921887
0.11233726712005403 0.9545154603485689 0.056168633560027006
0.48575252342344866 0.24287626171172427 0.7894019985716844
0.4004777133244753 0.827899293102909 0.20023885666223779
0.48360579825626965 0.7905444120471704 0.24180289912813469
0.5922751819477777 0.7397794586927233 0.2961375909738887
0.9999999999985895 0.5445378127845576 0.49999999999929434
0.029128786223406904 0.9891858882638618 0.014564393111703445
0.006481393111004906 0.9977527840366609 0.003240696555502453
0.05360166558996012 0.9802540552225478 0.026800832794980065
9.854096454110435e-06 0.9999975943958207 4.927048227055219e-06


{'gene1': 3.6678257823421223,
 'gene10': 0.53554191477264146,
 'gene11': 1.7688216795213752e-12,
 'gene12': 2.1817391980275018,
 'gene13': 2.7223781458865903,
 'gene14': 1.9300417447140998,
 'gene15': 4.420350444922196,
 'gene2': -0.058985912204876582,
 'gene3': -1.2243939880406665,
 'gene4': 0.44160051996547278,
 'gene5': 0.27142969322657989,
 'gene6': 1.587774820610055,
 'gene7': -0.69708032974689327,
 'gene8': 0.84076836409598821,
 'gene9': 0.70051490808337613}

In [9]:
for idx, item in enumerate(df.columns.values):
    print (idx, item)

0 phenotype
1 gene1
2 gene2
3 gene3
4 gene4
5 gene5
6 gene6
7 gene7
8 gene8
9 gene9
10 gene10
11 gene11
12 gene12
13 gene13
14 gene14
15 gene15


In [10]:
startp=[0,6,8,9,1,4,3]
for j, i in enumerate(startp):
    print (j,i, startp[j])

0 0 0
1 6 6
2 8 8
3 9 9
4 1 1
5 4 4
6 3 3


In [7]:
df.to_csv("data/df5000.csv")

In [8]:
counts.to_csv("data/counts.csv")

In [121]:
# bug in it, fixed in function "toy_multicnv_multicausal"
def toy_multi_gene_cnv(p, n_cnv, n_max=10, causal=5, const1=1, const2=0.1, seed=1):
    '''A region with multiple genes and CNVs. CNVs may overlap or not overlap. Only one causal gene.'''
    np.random.seed(seed)
    cnv_len = np.random.choice((np.random.geometric(p, size=n_cnv) - 1), n_cnv, replace=False)
    cnv_len = cnv_len[cnv_len <= 10].tolist()
    start_pos = [np.random.choice(range(n_max+1-i)) if i!=0 else -1 for i in cnv_len]
    ptn_ls = []
    for j,i in enumerate(start_pos):
        if i == -1:
            ptn = [0]*n_max
        elif i == 0:
            ptn = [0]*(i-1) + [1]*cnv_len[j] + [0]*(n_max-cnv_len[j]-i)
        else:
            ptn = [0]*(i-1) + [1]*cnv_len[j] + [0]*(n_max-cnv_len[j]-(i-1))
        ptn_ls.append(ptn)
    mat = [[np.random.normal(0,1)] + line for line in ptn_ls]
    config = []
    for line in mat:
##         config.append(line[0]+const1) if line[causal-1] == 1 else config.append(line[0])
        if line[causal] == 1:
            line[0] = line[0] + const1
        config.append(line[0])
        
#       the longer the CNV is, the less common it is, and larger OR
        line[0] = line[0] + line[1:].count(1)*const2
        
##         line[0] = 1 if line[0] > np.median(config) else 0
        if line[0] >= np.median(config):
            line[0] = 1
        else:
            line[0] = 0
        
    mat = np.matrix(mat)
    df = pd.DataFrame(mat, columns = ["phenotype"] + ["gene{}".format(i+1) for i in range(mat.shape[1]-1)])
    counts = df.groupby(["gene{}".format(i+1) for i in range(mat.shape[1]-1)] + ["phenotype"]).size()
    for col in df.columns:
        df[col] = df[col].astype(np.uint8)
    output = feather.write_dataframe(df, "data/toy_n{}_p{}_causal{}_const{}_{}.feather"
                                     .format(len(cnv_len), p, causal, const1, const2))
    return df, counts

In [123]:
df1, counts1 = toy_multi_gene_cnv(p=0.6, n_cnv=20, const1=1.0, const2=0.05)
# print (df["phenotype"].tolist())